In [4]:
%pip install pyautogen

Note: you may need to restart the kernel to use updated packages.


In [5]:
from typing_extensions import Annotated

import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST.json")
llm_config = {"config_list": config_list}

In [6]:
task = """KB국민은행 주가 추이에 대해 에 대해 간결하지만 매력적인 블로그 게시물을 작성해줘"""

In [7]:
writer = autogen.AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    system_message="""
    귀하는 통찰력 있고 매력적인 기사로 유명한 전문 작가입니다.
    복잡한 개념을 설득력 있는 이야기로 전환합니다.
    사용자의 피드백을 바탕으로 콘텐츠의 품질을 개선해야 합니다.
    """,
)

user_proxy = autogen.UserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

critic = autogen.AssistantAgent(
    name="Critic",
    llm_config={"config_list": config_list},
    system_message="""
    귀하는 기준에 대한 철저함과 헌신으로 유명한 비평가입니다.
    귀하의 임무는 콘텐츠에 유해한 요소나 규정 위반 사항이 있는지 면밀히 검토하여 다음과 같은 사항을 확인하는 것입니다.
    모든 자료가 필수 가이드라인에 부합하는지 확인합니다.
    코드의 경우
    """,
)

In [8]:
def reflection_message(recipient, messages, sender, config):
    print("Reflecting...", "yellow")
    return f"다음 글에 대해 반성하고 비평을 제공합니다. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"


user_proxy.register_nested_chats(
    [{"recipient": critic, "message": reflection_message, "summary_method": "last_msg", "max_turns": 1}],
    trigger=writer,  # condition=my_condition,
)

res = user_proxy.initiate_chat(recipient=writer, message=task, max_turns=2, summary_method="last_msg")

User (to Writer):

KB국민은행 주가 추이에 대해 에 대해 간결하지만 매력적인 블로그 게시물을 작성해줘

--------------------------------------------------------------------------------
Writer (to User):

### KB국민은행 주가 추이: 금융의 심장을 읽다

최근 금융 시장이 촉망하는 주식 중 하나인 KB국민은행의 주가 동향에 대해 알아보고자 합니다. 대한민국을 대표하는 금융기관 KB국민은행은 그 신뢰성과 안정성으로 투자자들의 지속적인 관심을 받아왔습니다.

#### 안정적인 성장의 여정

KB국민은행의 주가는 전반적으로 꾸준한 상승세를 보여왔습니다. 경제 불안정 시기에도 상대적으로 안정적인 주가 흐름을 보이며 금융시장의 든든한 버팀목 역할을 했습니다. 이는 KB국민은행이 구축한 탄탄한 재무구조와 리스크 관리 능력 덕분인데요, 자산 건전성 지표가 업계 평균을 웃돌고 있습니다.

#### 디지털 전환과 혁신의 발걸음

한편, 최근 KB국민은행은 디지털 금융 플랫폼 강화에 박차를 가하고 있습니다. 비대면 서비스의 확대와 인공지능(AI)을 활용한 맞춤형 금융 서비스를 도입하여 젊은 세대의 마음을 사로잡고 있습니다. 이러한 디지털 혁신은 주가에도 긍정적인 영향을 미치며 새로운 성장 동력으로 자리 잡고 있습니다.

#### 글로벌 도전과 다양화 전략

KB국민은행은 국내 시장에 안주하지 않고 글로벌 시장으로의 확장을 추진하고 있습니다. 동남아시아를 비롯한 해외 진출이 본격화되면서, 수익 다변화와 글로벌 경쟁력 강화를 통한 추가적인 주가 상승이 예상됩니다. 또한, ESG(환경, 사회, 지배구조) 경영에 집중하여 지속 가능한 발전을 도모하고 있는 점도 주목할 만합니다.

#### 향후 전망

경제 예측은 쉽지 않지만, KB국민은행의 견고한 재무 상태와 전략적 방향성은 긍정적인 신호로 여겨질 수 있습니다. 금융시장 전문가들은 KB국민은행이 지속적인 혁신과 글로벌 확장을 통해 보다 안